Stability analysis in the theta model

In [4]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.interpolate import interp1d
from generate_figures import get_antiphase, get_antiphase_fulltheta

mod = np.mod

# load dat file
namexx = 'hxx_fixed_a1=0.1_b1=1.0_c1=1.1_a2=0.1_b2=1.0_c2=1.1_eps=0.01_mux=1.0_muy=1.0.dat'
namexy = 'hxy_fixed_a1=0.1_b1=1.0_c1=1.1_a2=0.1_b2=1.0_c2=1.1_eps=0.01_mux=1.0_muy=1.0.dat'
nameyx = 'hyx_fixed_a1=0.1_b1=1.0_c1=1.1_a2=0.1_b2=1.0_c2=1.1_eps=0.01_mux=1.0_muy=1.0.dat'
nameyy = 'hyy_fixed_a1=0.1_b1=1.0_c1=1.1_a2=0.1_b2=1.0_c2=1.1_eps=0.01_mux=1.0_muy=1.0.dat'

#namexx = 'hxx_a1=0.5_b1=7.0_c1=6.5_a2=1.1_b2=25.0_c2=25.1_eps=0.01_mux=1.0_muy=1.0.dat'
#namexy = 'hxy_a1=0.5_b1=7.0_c1=6.5_a2=1.1_b2=25.0_c2=25.1_eps=0.01_mux=1.0_muy=1.0.dat'
#nameyx = 'hyx_a1=0.5_b1=7.0_c1=6.5_a2=1.1_b2=25.0_c2=25.1_eps=0.01_mux=1.0_muy=1.0.dat'
#nameyy = 'hyy_a1=0.5_b1=7.0_c1=6.5_a2=1.1_b2=25.0_c2=25.1_eps=0.01_mux=1.0_muy=1.0.dat'

datxx=np.loadtxt(namexx);datxy=np.loadtxt(namexy);datyx=np.loadtxt(nameyx);datyy=np.loadtxt(nameyy)
per = datxx[-1,0]

hxx_interp = interp1d(datxx[:,0],datxx[:,1])
hyx_interp = interp1d(datyx[:,0],datyx[:,1])

def hxx(x):
    #print x,'hxxin'
    x = mod(x,per)
    return hxx_interp(x)

def hxy(x,tau):
    x = mod(x,per)
    hxy_interp = interp1d(datxy[:,0],datxy[:,1]/tau)
    return hxy_interp(x)

def hyx(x):
    x = mod(x,per)
    return hyx_interp(x)

def hyy(x,tau):
    x = mod(x,per)
    hyy_interp = interp1d(datyy[:,0],datyy[:,1]/tau)
    return hyy_interp(x)


tau = 2.62


def rhs(px,py,pz,tau):
    """
    RHS of theta model phase equations
    """
    
    rhs_x=hxx(-px)-hxx(px)+hxy(-px+pz,tau)-hxy(pz,tau)+hxy(py-px+pz,tau)-hxy(py+pz,tau)
    rhs_y=hyx(-py-pz)-hyx(-pz)+hyy(-py,tau)+hyx(px-py-pz)-hyx(px-pz)-hyy(py,tau)
    rhs_z=hyx(-pz)+hyx(px-pz)+hyy(py,tau)-hxy(pz,tau)-hxx(px)-hxy(py+pz,tau)
    
    return np.array([rhs_x,rhs_y,rhs_z])


    
dhxx_interp = interp1d(datxx[:,0],np.gradient(hxx(datxx[:,0]),np.diff(datxx[:,0])[0]))
dhyx_interp = interp1d(datyx[:,0],np.gradient(hyx(datyx[:,0]),np.diff(datyx[:,0])[0]))


def dhxx(x):
    x = mod(x,per)
    return dhxx_interp(x)

def dhxy(x,tau):
    x = mod(x,per)
    dhxy_interp = interp1d(datxy[:,0],np.gradient(hxy(datxy[:,0],tau),np.diff(datxy[:,0])[0]))
    return dhxy_interp(x)

def dhyx(x):
    x = mod(x,per)
    return dhyx_interp(x)

def dhyy(x,tau):
    x = mod(x,per)
    dhyy_interp = interp1d(datyy[:,0],np.gradient(hyy(datyy[:,0],tau),np.diff(datyy[:,0])[0]))
    return dhyy_interp(x)


def jac(px,py,pz):
    """
    Jacobian matrix
    """
    j11 = -dhxx(mod(-px,per))-dhxx(mod(px,per))-dhxy(mod(-px+pz,per))-dhxy(mod(py-px+pz,per))
    j12 = dhxy(mod(py-px+pz,per))-dhxy(mod(py+pz,per))
    j13 = dhxy(mod(-px+pz,per))-dhxy(mod(pz,per))+dhxy(mod(py-px+pz,per))-dhxy(mod(py+pz,per))
    
    j21 = dhyx(mod(px-py+pz,per))-dhyx(mod(px-pz,per))
    j22 = -dhyx(mod(-py-pz,per))-dhyx(mod(px-py-pz,per))-dhyy(mod(-py,per))-dhyy(mod(py,per))
    j23 = -dhyx(mod(-py-pz,per))+dhyx(mod(-pz,per))-dhyx(mod(px-py-pz,per))+dhyx(mod(px-pz,per))
    
    j31 = dhyx(mod(px-pz,per))-dhxx(mod(px,per))
    j32 = dhyy(mod(py,per))-dhxy(mod(py+pz,per))
    j33 = -dhyx(mod(-pz,per))-dhyx(mod(px-pz,per))-dhxy(mod(pz,per))-dhxy(mod(py+pz,per))
    
    return np.array([[j11,j12,j13],
                     [j21,j22,j23],
                     [j31,j32,j33]])

antip_val2 = get_antiphase_fulltheta(rhs,datxx[0,0],datxx[-1,0],choice='aaa',tau=1)

print antip_val2,'antip_val2'
antip_val = get_antiphase(hxx,datxx[0,0],datxx[-1,0])
#print antip_val

if False:
    fig = plt.figure()
    ax = fig.add_subplot(111)
    domxx = datxx[:,0]
    domxy = datxy[:,0]
    
    ax.plot(domxx,hxx(domxx),label='hxx')
    ax.plot(domxy,hxy(domxy),label='hxy')
    
    if True:
        ax.plot(domxx,hxx(mod(-domxx,per)),color='black')
        ax.plot(domxx,hxx(mod(domxx,per)),color='black')
        #hxx(mod(-px,per))-hxx(mod(px,per))+hxy(mod(-px+pz,per))-hxy(mod(pz,per))+hxy(mod(py-px+pz,per))-hxy(mod(py+pz,per))
    
    ax.scatter(antip_val,0)
    ax.legend()
    plt.show()


#print datxx[-1,0]/2
px = antip_val
py = 0
pz = antip_val#np.linspace(0,per,per00)

#jii=-dhxx[0]-dhxy[0]
#jjj=-dhyx[0]-dhyy[0]
#jzz=-dhyx[0]-dhxy[0]

#print 'rhs',rhs(px,py,pz)
#print 'eig',np.linalg.eig(jac(px,py,pz))[0]
#print jii,jjj,jzz



if False:
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(pz,rhs_z)

    plt.show()

1.85164181622 antip_val2


Below I fix/plot the bifurcation diagrams for the single antiphase solutions

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from thetaslowmod_lib import collect_disjoint_branches

from scipy.interpolate import interp1d

if True:
    py0a0 = np.loadtxt('py0a0_bifurcation.dat')
    val,ty = collect_disjoint_branches(py0a0,remove_isolated=True,isolated_number=5,remove_redundant=False,redundant_threshold=.2,N=2,fix_reverse=False)

    #print val.keys()

    plt.figure()
    #keyy = 'br0'
    #print val[keyy]
    #plt.plot(val[keyy][:,0],val[keyy][:,3])

    for key in val.keys():
        plt.plot(val[key][:,0],val[key][:,3],label=key)

    plt.legend()
    plt.show()

if False:
    
    pxa00 = np.loadtxt('pxa00_bifurcation.dat')
    
    """
    val,ty = collect_disjoint_branches(pxa00,remove_isolated=True,
                                       isolated_number=5,
                                       remove_redundant=False,
                                       redundant_threshold=.2,
                                       N=2,fix_reverse=False)
    """
    
    plt.plot(pxa00[:43,0],pxa00[:43,1])
    plt.plot(pxa00[43:,0],pxa00[43:,1])
    
    plt.legend()
    plt.show()


    """
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(pz00a[:,3],pz00a[:,6])

    plt.show()
    """


if False:
    pz00a = np.loadtxt('pz00a_bifurcation.dat')
    val,ty = collect_disjoint_branches(pz00a,remove_isolated=True,isolated_number=5,remove_redundant=False,redundant_threshold=.2,N=2,fix_reverse=False)

    #print val.keys()

    plt.figure()
    #keyy = 'br0'
    #print val[keyy]
    #plt.plot(val[keyy][:,0],val[keyy][:,3])

    for key in val.keys():
        plt.plot(val[key][:,0],val[key][:,3],label=key)

    plt.legend()
    plt.show()


    """
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(pz00a[:,3],pz00a[:,6])

    plt.show()
    """

['br9', 'br8', 'br7', 'br6', 'br5', 'br4', 'br3', 'br2', 'br1', 'br0', 'br13', 'br12', 'br11', 'br10', 'br17', 'br16', 'br15', 'br14', 'br18']


Below I fix/plot the bifurcation diagrams for the single antiphase solutions

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import proj3d
from thetaslowmod_lib import collect_disjoint_branches,collect3d_colorgrad,collect

pyz0aa = np.loadtxt('pyz0aa_bifurcation.dat')

val,ty = collect_disjoint_branches(pyz0aa,remove_isolated=True,
                                   isolated_number=10,
                                   remove_redundant=False,
                                   redundant_threshold=.2,
                                   N=2,fix_reverse=True)



fig = plt.figure(figsize=(6,6))
ax1 = fig.add_subplot(111, projection='3d')

per=3.70156211872
#per = 

xlo = 10
xhi = -10

ylo = 10
yhi = -10

print val.keys()

for key in val.keys():
    if (key == 'br5') or \
        (key == 'br4') or \
        (key == 'br2') or \
        (key == 'br0'):
        muy = val[key][:,0]
        pya = np.mod(val[key][:,2]+per/2.,per)-per/2.
        pza = np.mod(val[key][:,3]+per/2.,per)-per/2.
        
        muy = muy[muy<1.5]
        pya = py[muy<1.5]
        pza = pz[muy<1.5]
        
        
        # given python branch, just determine stability along that one
        stbl = np.zeros(len(muy))
        
        for i in range(len(muy)):
            tau = muy[i]
            px = 0
            py = pya[i]
            pz = pza[i]
            
            #print px,py,pz,tau
            eigs = np.linalg.eig(jac(px,py,pz,tau))[0]
            # if real part positive, mark unstable
            
            if np.sum(np.real(eigs)>0):
                stbl[i] = 1
            else:
                stbl[i] = -1
            #print stbl1[i]
        
        # plot stable
        ax1.plot(py[stbl<0],
                 (mod(pz+per/2.,per)-per/2.)[stbl<0],
                 tau[stbl<0],
                 label=key,color='black',lw=2,zorder=2)
        # plot unstable
        ax1.plot(py[stbl>0],
                 (mod(pz+per/2.,per)-per/2.)[stbl>0],
                 tau[stbl>0],
                 label=key,color='black',lw=2,zorder=2,ls='--',dashes=(4,2))
        
        #ax1.plot(py,pz,muy,color='black',lw=2)
        
        if key == 'br4':
            
            #pltobj = collect3d_colorgrad(py,pz,muy,
            #                             lwstart=4,lwend=5,
            #                            cmapmin=0,cmapmax=.9)
            proj_fig = collect(py,pz,use_nonan=False,lwstart=1,lwend=1,
                              cmapmin=0.5,cmapmax=0.5,cmap='gray')
            
        elif key == 'br5':
            #plotobj = collect3d_colorgrad(py,pz,muy,
            #                              lwstart=1,lwend=4,
            #                             cmapmin=0,cmapmax=.9)
            proj_fig = collect(py,pz,use_nonan=False,lwstart=1,lwend=1,
                              cmapmin=0.5,cmapmax=0.5,cmap='gray')
            
        elif key == 'br2':
            #plotobj = collect3d_colorgrad(py,pz,muy,
            #                              lwstart=5,lwend=1,
            #                             cmapmin=0,cmapmax=1)
            proj_fig = collect(py,pz,use_nonan=False,lwstart=5,lwend=1,
                              cmapmin=0.5,cmapmax=0.5,cmap='gray')
            
        else:
            #plotobj = collect3d_colorgrad(py,pz,muy)
            proj_fig = collect(py,pz,use_nonan=False,lwstart=1,lwend=1,
                              cmapmin=0.5,cmapmax=0.5,cmap='gray')
        
        ax1.add_collection3d(proj_fig,zdir='z',zs=1)
        #ax1.add_collection3d(plotobj)
        
        # get x bounds
        if np.nanmin(py) < xlo:
            xlo = np.nanmin(py)
        if np.nanmax(py) > xhi:
            xhi = np.nanmax(py)
            
        # get y bounds
        if np.nanmin(pz) < ylo:
            ylo = np.nanmin(pz)
        if np.nanmax(pz) > yhi:
            yhi = np.nanmax(pz)
            
ax1.set_xlim(xlo,xhi)
ax1.set_ylim(ylo,yhi)
ax1.set_zlim(1,1.5)
    
ax1.set_xlabel('py')
ax1.set_ylabel('pz')
ax1.set_zlabel('muy')
    
plt.show()

['br5', 'br4', 'br3', 'br2', 'br1', 'br0']
['br5', 'br4', 'br3', 'br2', 'br0']


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:37: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 60 but corresponding boolean dimension is 43
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:38: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 60 but corresponding boolean dimension is 43


NameError: name 'jac' is not defined

Below, check Jacobian eigenvalues and fixed points

In [60]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import proj3d
from thetaslowmod_lib import *

mod = np.mod

# load dat file
namexx = 'hxx_fixed_a1=0.1_b1=1.0_c1=1.1_a2=0.1_b2=1.0_c2=1.1_eps=0.01_mux=1.0_muy=1.0.dat'
namexy = 'hxy_fixed_a1=0.1_b1=1.0_c1=1.1_a2=0.1_b2=1.0_c2=1.1_eps=0.01_mux=1.0_muy=1.0.dat'
nameyx = 'hyx_fixed_a1=0.1_b1=1.0_c1=1.1_a2=0.1_b2=1.0_c2=1.1_eps=0.01_mux=1.0_muy=1.0.dat'
nameyy = 'hyy_fixed_a1=0.1_b1=1.0_c1=1.1_a2=0.1_b2=1.0_c2=1.1_eps=0.01_mux=1.0_muy=1.0.dat'

#namexx = 'hxx_a1=0.5_b1=7.0_c1=6.5_a2=1.1_b2=25.0_c2=25.1_eps=0.01_mux=1.0_muy=1.0.dat'
#namexy = 'hxy_a1=0.5_b1=7.0_c1=6.5_a2=1.1_b2=25.0_c2=25.1_eps=0.01_mux=1.0_muy=1.0.dat'
#nameyx = 'hyx_a1=0.5_b1=7.0_c1=6.5_a2=1.1_b2=25.0_c2=25.1_eps=0.01_mux=1.0_muy=1.0.dat'
#nameyy = 'hyy_a1=0.5_b1=7.0_c1=6.5_a2=1.1_b2=25.0_c2=25.1_eps=0.01_mux=1.0_muy=1.0.dat'

datxx=np.loadtxt(namexx);datxy=np.loadtxt(namexy);datyx=np.loadtxt(nameyx);datyy=np.loadtxt(nameyy)
per = datxx[-1,0]

print 'last vals',datxx[-2:,1]

xlo = datxx[0,0]
xhi = datxx[-1,0]

hxx_interp = interp1d(datxx[:,0],datxx[:,1])
hyx_interp = interp1d(datyx[:,0],datyx[:,1])

def hxx(x):
    x = mod(x,per)
    return hxx_interp(x)

def hxy(x,tau):
    x = mod(x,per)
    hxy_interp = interp1d(datxy[:,0],datxy[:,1]/tau)
    return hxy_interp(x)

def hyx(x):
    x = mod(x,per)
    return hyx_interp(x)

def hyy(x,tau):
    x = mod(x,per)
    hyy_interp = interp1d(datyy[:,0],datyy[:,1]/tau)
    return hyy_interp(x)


def rhs(px,py,pz,tau):
    """
    RHS of theta model phase equations
    """
    
    rhs_x=hxx(-px)-hxx(px)+hxy(-px+pz,tau)-hxy(pz,tau)+hxy(py-px+pz,tau)-hxy(py+pz,tau)
    rhs_y=hyx(-py-pz)-hyx(-pz)+hyy(-py,tau)+hyx(px-py-pz)-hyx(px-pz)-hyy(py,tau)
    rhs_z=hyx(-pz)+hyx(px-pz)+hyy(py,tau)-hxy(pz,tau)-hxx(px)-hxy(py+pz,tau)
    
    return np.array([rhs_x,rhs_y,rhs_z])



dhxx_interp = interp1d(datxx[:,0],np.gradient(datxx[:,1],np.diff(datxx[:,0])[0]))
dhyx_interp = interp1d(datyx[:,0],np.gradient(datyx[:,1],np.diff(datyx[:,0])[0]))


def dhxx(x):
    x = mod(x,per)

    return dhxx_interp(x)

def dhxy(x,tau):
    x = mod(x,per)
    dhxy_interp = interp1d(datxy[:,0],np.gradient(datxy[:,1]/tau,np.diff(datxy[:,0])[0]))
    return dhxy_interp(x)

def dhyx(x):
    x = mod(x,per)
    return dhyx_interp(x)

def dhyy(x,tau):
    x = mod(x,per)
    dhyy_interp = interp1d(datyy[:,0],np.gradient(datyy[:,1]/tau,np.diff(datyy[:,0])[0]))
    return dhyy_interp(x)

if False:
    fig = plt.figure()
    ax = fig.add_subplot(111)
    x = np.linspace(0,per,100)
    #ax.plot(x,dhxx(x))
    #ax.plot(x,dhxy(x,5))
    ax.plot(x,dhyx(x))
    #ax.plot(x,dhyy(x,5))
    plt.show()

def jac(px,py,pz,tau):
    """
    Jacobian matrix
    """
    
    j11 = -dhxx(-px)-dhxx(px)-dhxy(-px+pz,tau)-dhxy(py-px+pz,tau)
    j12 = dhxy(py-px+pz,tau)-dhxy(py+pz,tau)
    j13 = dhxy(-px+pz,tau)-dhxy(pz,tau)+dhxy(py-px+pz,tau)-dhxy(py+pz,tau)
    
    j21 = dhyx(px-py+pz)-dhyx(px-pz)
    j22 = -dhyx(-py-pz)-dhyx(px-py-pz)-dhyy(-py,tau)-dhyy(py,tau)
    j23 = -dhyx(-py-pz)+dhyx(-pz)-dhyx(px-py-pz)+dhyx(px-pz)
    
    j31 = dhyx(px-pz)-dhxx(px)
    j32 = dhyy(py,tau)-dhxy(py+pz,tau)
    j33 = -dhyx(-pz)-dhyx(px-pz)-dhxy(pz,tau)-dhxy(py+pz,tau)
    
    return np.array([[j11,j12,j13],
                     [j21,j22,j23],
                     [j31,j32,j33]])


print np.linalg.eig(jac(1.673,0,1.675,1.132))[0], rhs(1.673,0,1.675,1.132)
print np.linalg.eig(jac(1.585,0,1.585,1.136))[0], rhs(1.585,0,1.585,1.136)
print
print np.linalg.eig(jac(1.79967,0.,1.80166,1.20875))[0], rhs(1.79967,0,1.80166,1.20875)
print np.linalg.eig(jac(1.75374895658,0.0,1.75374895658,1.20202020202))[0], rhs(1.75374895658,0.0,1.75374895658,1.20202020202)


print jac(1.79967,0.,1.80166,1.20875)
print jac(1.75374895658,0.0,1.75374895658,1.20202020202)

print dhyx(1.79967+1.80166),dhyx(1.79967-1.80166),mod(1.79967-1.80166,per),dhyx(3.699)
print dhyx(2*1.75374895658),dhyx(0)


if True:
    fig = plt.figure()
    ax = fig.add_subplot(111)
    x = np.linspace(0,per,100000)
    #ax.plot(x,dhxx(x))
    #ax.plot(x,dhxy(x,5))
    ax.plot(x,dhyx(x))
    ax.plot(x,hyx(x))
    #ax.plot(x,dhyy(x,5))
    #ax.scatter(3.7,dhyx(3.7))
    #ax.scatter(3.699,dhyx(3.699))
    #ax.scatter(3.69,dhyx(3.69))
    plt.show()



last vals [ 0.00507172  0.00253586]
[  0.76163626 -14.13531464 -13.01353239] [  1.07897526e-01   1.71370181e-18   8.44132481e-02]
[  1.87752956 -14.85855935 -12.58477349] [ 0.09999878  0.          0.09999878]

[ -1.12003518 -12.55129652 -12.44576828] [  1.00953924e-01  -1.31418167e-18   7.87247235e-02]
[ -0.7546879  -12.84164607 -12.44711306] [ 0.10083152  0.          0.10083152]
[[-26.11895323  12.44667582  24.89335164]
 [  0.09994807 -12.44576828   0.        ]
 [-13.67516774  12.4466949   12.44762153]]
[[-26.05489813  12.45283861  24.90567721]
 [  0.37044146 -12.44711306   0.        ]
 [-13.60778507  12.45283861  12.45856416]]
-6.75008332219 -6.8500313875 3.69957211872 -6.84999963196
-6.4796545172 -6.85009598125
